In [3]:
#from dotenv import load_dotenv
#import os 

In [4]:
#from alpha_vantage.timeseries import TimeSeries
#import pandas as pd

#load_dotenv()

#api_key = os.environ.get('stocks_api') # Replace this with your actual API key
#ts = TimeSeries(key=api_key, output_format='pandas')

# Fetch daily adjusted stock prices; adjust the symbol as needed
#data, meta_data = ts.get_daily(symbol='TSLA', outputsize='full')

#print(data.head())

            1. open  2. high    3. low  4. close    5. volume
date                                                         
2024-04-30   186.98   190.95  182.8401    183.28  127031787.0
2024-04-29   188.42   198.87  184.5400    194.05  243869678.0
2024-04-26   168.85   172.12  166.3700    168.29  109815725.0
2024-04-25   158.96   170.88  158.3600    170.18  126427521.0
2024-04-24   162.84   167.97  157.5100    162.13  181178020.0


In [5]:
#data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3483 entries, 2024-04-30 to 2010-06-29
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   1. open    3483 non-null   float64
 1   2. high    3483 non-null   float64
 2   3. low     3483 non-null   float64
 3   4. close   3483 non-null   float64
 4   5. volume  3483 non-null   float64
dtypes: float64(5)
memory usage: 163.3 KB


In [6]:
#meta_data

{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'TSLA',
 '3. Last Refreshed': '2024-04-30',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

In [6]:
# Define your file path and name
file_path = '/Users/manos/Documents/BDS/MLops_mod/TSLA_stock_price.csv'  # Customize the path and filename

# Save the DataFrame to CSV
data.to_csv(file_path)

print(f"Data saved to {file_path}")


Data saved to /Users/manos/Documents/BDS/MLops_mod/TSLA_stock_price.csv


In [7]:
# Load and display the data from CSV to confirm
tsla_df = pd.read_csv(file_path)
print(tsla_df.head())


         date  1. open  2. high  3. low  4. close    5. volume
0  2024-04-29   188.42   198.87  184.54    194.05  243869678.0
1  2024-04-26   168.85   172.12  166.37    168.29  109815725.0
2  2024-04-25   158.96   170.88  158.36    170.18  126427521.0
3  2024-04-24   162.84   167.97  157.51    162.13  181178020.0
4  2024-04-23   143.33   147.26  141.11    144.68  124545104.0


In [8]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store()


Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549016
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
#import re 

In [10]:
#def clean_column_name(name):
    # Remove all non-letter characters
    #cleaned_name = re.sub(r'[^a-zA-Z]', '', name)
    #return cleaned_name


In [11]:
#tsla_df

,date,1. open,2. high,3. low,4. close,5. volume
0,2024-04-29,188.42,198.8700,184.54,194.05,243869678.0
1,2024-04-26,168.85,172.1200,166.37,168.29,109815725.0
2,2024-04-25,158.96,170.8800,158.36,170.18,126427521.0
3,2024-04-24,162.84,167.9700,157.51,162.13,181178020.0
4,2024-04-23,143.33,147.2600,141.11,144.68,124545104.0
...,...,...,...,...,...,...
3477,2010-07-06,20.00,20.0000,15.83,16.11,6866900.0
3478,2010-07-02,23.00,23.1000,18.71,19.20,5139800.0
3479,2010-07-01,25.00,25.9200,20.27,21.96,8218800.0
3480,2010-06-30,25.79,30.4192,23.30,23.83,17187100.0


In [12]:
# Assuming 'tsla_df' is your DataFrame
#tsla_df.columns = [clean_column_name(col) for col in tsla_df.columns]


In [13]:
#print(tsla_df.columns)


Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')


In [14]:
# Define a feature group
tesla_fg = fs.get_or_create_feature_group(
    name="tsla_stock",
    description="Tesla stock dataset from alpha vantage",
    version=1,
    primary_key=["date"],
    online_enabled=True,
)

In [15]:
tesla_fg.insert(tsla_df, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/3482 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: tsla_stock_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549016/jobs/named/tsla_stock_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x162ac3e50>, None)

In [18]:
# Create feature group for historical news data
news_df = pd.read_csv('/Users/manos/Documents/BDS/MLops_mod/news_articles.csv')

news_sentiment_fg = fs.get_or_create_feature_group(
    name='news_sentiment',
    description='News sentiment from Polygon',
    version=1,
    primary_key=['date'],
    online_enabled=True,
)

news_sentiment_fg.insert(news_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/549016/fs/544838/fg/766341


Uploading Dataframe: 0.00% |          | Rows 0/712 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: news_sentiment_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549016/jobs/named/news_sentiment_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x164180710>, None)